In [2]:
import os
import sys
from pathlib import Path

import einops
import plotly.express as px
import torch as t
from IPython.display import display
from ipywidgets import interact
from jaxtyping import Bool, Float, jaxtyped
from torch import Tensor
from typeguard import typechecked as typechecker

# Make sure exercises are in the path
chapter = r"chapter0_fundamentals"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part1_ray_tracing"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

import part1_ray_tracing.tests as tests
from part1_ray_tracing.utils import (
    render_lines_with_plotly,
    setup_widget_fig_ray,
    setup_widget_fig_triangle,
)
from plotly_utils import imshow

MAIN = __name__ == "__main__"

In [3]:
def make_rays_1d(num_pixels: int, y_limit: float) -> t.Tensor:
    '''
    num_pixels: The number of pixels in the y dimension. Since there is one ray per pixel, this is also the number of rays.
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both endpoints.

    Returns: shape (num_pixels, num_points=2, num_dim=3) where the num_points dimension contains (origin, direction) and the num_dim dimension contains xyz.

    Example of make_rays_1d(9, 1.0): [
        [[0, 0, 0], [1, -1.0, 0]],
        [[0, 0, 0], [1, -0.75, 0]],
        [[0, 0, 0], [1, -0.5, 0]],
        ...
        [[0, 0, 0], [1, 0.75, 0]],
        [[0, 0, 0], [1, 1, 0]],
    ]
    '''
    zeroes = t.zeros((num_pixels,2, 3))

    #y_limit = 1.0
    #num_pixels = 9.0
    #num_steps = (num_pixels - 1)
    #step_size = (2 * y_limit) / num_steps


    #t.arange(-y_limit, y_limit + step_size, (2 * y_limit) / num_steps, out=zeroes[:, 1, 1])
    t.linspace(-y_limit, y_limit, int(num_pixels),  out=zeroes[:, 1, 1])
    zeroes[:, 1, 0] = 1

    return zeroes
    

rays1d = make_rays_1d(9, 10.0)

print(rays1d)

fig = render_lines_with_plotly(rays1d)

tensor([[[  0.0000,   0.0000,   0.0000],
         [  1.0000, -10.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -7.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -5.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -2.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   0.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   2.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   5.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   7.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  10.0000,   0.0000]]])


In [4]:
segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]], 
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]], 
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])

fig = render_lines_with_plotly(t.cat((rays1d, segments)))

In [5]:
def intersect_ray_1d(ray: t.Tensor, segment: t.Tensor) -> bool:
    """
    ray: shape (n_points=2, n_dim=3)  # O, D points
    segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

    Return True if the ray intersects the segment.
    """
    # [2, 3] -> [2, 2]
    ray = ray[..., :2]
    # [2, 3] -> [2, 2]
    segment = segment[..., :2]

    # [2]
    origin = ray[0]
    # [2]
    direction = ray[1]
    # [2]
    l1 = segment[0]
    # [2]
    l2 = segment[1]

    l1_minus_l2 = l1 - l2

    # [2, 2]
    lhs = t.stack((direction, l1_minus_l2), dim=1)
    # [2]
    rhs = l1 - origin
    try:
        solution_vector = t.linalg.solve(lhs, rhs)
    except:
        return False

    u = solution_vector[0]
    v = solution_vector[1]

    return u >= 0.0 and v >= 0.0 and v <= 1.0


tests.test_intersect_ray_1d(intersect_ray_1d)
tests.test_intersect_ray_1d_special_case(intersect_ray_1d)

All tests in `test_intersect_ray_1d` passed!
All tests in `test_intersect_ray_1d_special_case` passed!


/Users/mjs/arena/.venv/lib/python3.11/site-packages/jaxtyping/_decorator.py:73: InstrumentationWarning:

instrumentor did not find the target function -- not typechecking part1_ray_tracing.solutions.triangle_ray_intersects



In [6]:
def intersect_rays_1d(
    rays: Float[Tensor, "nrays 2 3"], segments: Float[Tensor, "nsegments 2 3"]
) -> Bool[Tensor, "nrays"]:
    """
    For each ray, return True if it intersects any segment.
    """
    # [nrays, 2, 2]
    rays = rays[..., :2]
    # [nsegments, 2, 2]
    segments = segments[..., :2]

    # [nrays, 2]
    origins = rays[:, 0]
    # [nrays, 2]
    directions = rays[:, 1]
    # [nsegments, 2]
    l1s = segments[:, 0]
    # [nsegments, 2]
    l2s = segments[:, 1]
    # [nsegments, 2]
    l1s_minus_l2s = l1s - l2s

    n_segments = segments.shape[0]
    n_rays = rays.shape[0]
    # [nrays, nsegments, 2]
    repeated_ray_directions = einops.repeat(directions, 'a b -> a n_segments b', n_segments=n_segments)
    # [nrays, nsegments, 2]
    repeated_l1s_minus_l2s = einops.repeat(l1s_minus_l2s, 'a b -> c a b', c=n_rays)

    # [nrays, nsegments, 2, 2]
    lhs = t.stack((repeated_ray_directions, repeated_l1s_minus_l2s), dim=3)

    # [nrays, nsegments, 2]
    repeated_origins = einops.repeat(origins, 'a b -> a n_segments b', n_segments=n_segments)

    rhs = l1s - repeated_origins

    # TODO: Look for rhs matrices with 0 determinant 
    matrices_with_zero_determinant = t.linalg.det(lhs).abs() < 1e-8

    lhs[matrices_with_zero_determinant] = t.eye(2)

    # [nrays, nsegments, 2]
    solutions = t.linalg.solve(lhs, rhs)

    u_geq_0 = solutions[..., 0] >= 0.0
    v_in_range = (solutions[..., 1] >= 0.0) & (solutions[..., 1] <= 1.0)
    v_was_singular = matrices_with_zero_determinant
    return (u_geq_0 & v_in_range & ~v_was_singular).any(dim=1)


tests.test_intersect_rays_1d(intersect_rays_1d)
tests.test_intersect_rays_1d_special_case(intersect_rays_1d)

All tests in `test_intersect_rays_1d` passed!
All tests in `test_intersect_rays_1d_special_case` passed!


In [7]:
def make_rays_2d(num_pixels_y: int, num_pixels_z: int, y_limit: float, z_limit: float) -> Float[Tensor, "nrays 2 3"]:
    """
    num_pixels_y: The number of pixels in the y dimension
    num_pixels_z: The number of pixels in the z dimension

    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both.
    z_limit: At x=1, the rays should extend from -z_limit to +z_limit, inclusive of both.

    Returns: shape (num_rays=num_pixels_y * num_pixels_z, num_points=2, num_dims=3).
    """

    num_rays = num_pixels_y * num_pixels_z
    rays = t.zeros((num_rays, 2, 3))
    assert rays.shape == (num_rays, 2, 3)

    z_linspace = t.linspace(-z_limit, z_limit, num_pixels_z)

    rays[:, 1, 2] = einops.repeat(z_linspace, 'a -> (num_pixels_z a)', num_pixels_z=num_pixels_z)

    y_linspace = t.linspace(-y_limit, y_limit, num_pixels_y)
    rays[:, 1, 1] = einops.repeat(y_linspace, 'a ->  (a num_pixels_y)', num_pixels_y=num_pixels_y)

    rays[:, 1, 0] = 1.0

    return rays


rays_2d = make_rays_2d(10, 10, 0.3, 0.3)
render_lines_with_plotly(rays_2d)

In [21]:
Point = Float[Tensor, "points=3"]


def triangle_ray_intersects(A: Point, B: Point, C: Point, O: Point, D: Point) -> bool:
    """
    A: shape (3,), one vertex of the triangle
    B: shape (3,), second vertex of the triangle
    C: shape (3,), third vertex of the triangle
    O: shape (3,), origin point
    D: shape (3,), direction point

    Return True if the ray and the triangle intersect.
    """

    lhs = t.stack((-D, B-A, C-A), dim=1)

    rhs = O-A

    try:
        solution_vector = t.linalg.solve(lhs, rhs)
    except:
        return False

    s, u, v = solution_vector

    # check that u and v are within the triangle
    u_and_v_within_triangle = ((u >= 0.0 and u <= 1.0) and (v >= 0.0 and v <= 1.0)).item()
    return u_and_v_within_triangle


tests.test_triangle_ray_intersects(triangle_ray_intersects)


All tests in `test_triangle_ray_intersects` passed!


In [ ]:
def raytrace_triangle(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"], triangle: Float[Tensor, "trianglePoints=3 dims=3"]
) -> Bool[Tensor, "nrays"]:
    """
    For each ray, return True if the triangle intersects that ray.
    """
    raise NotImplementedError()


A = t.tensor([1, 0.0, -0.5])
B = t.tensor([1, -0.5, 0.0])
C = t.tensor([1, 0.5, 0.5])
num_pixels_y = num_pixels_z = 15
y_limit = z_limit = 0.5

# Plot triangle & rays
test_triangle = t.stack([A, B, C], dim=0)
rays2d = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
triangle_lines = t.stack([A, B, C, A, B, C], dim=0).reshape(-1, 2, 3)
render_lines_with_plotly(rays2d, triangle_lines)

# Calculate and display intersections
intersects = raytrace_triangle(rays2d, test_triangle)
img = intersects.reshape(num_pixels_y, num_pixels_z).int()
imshow(img, origin="lower", width=600, title="Triangle (as intersected by rays)")